In [1]:
### import necessary libraries
import ee
import json
import geemap
import sklearn
import numpy as np
import pandas as pd
import geopandas as gpd
from pprint import pprint
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize(project="ee-franciscofurey")

In [3]:
# Select images by predefined dates
before_start = '2018-07-15'
before_end = '2018-08-10'
after_start = '2018-08-10'
after_end = '2018-08-23'

admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
ernakulam = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Ernakulam'))
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
geometry = ernakulam.geometry()

Map = geemap.Map()
Map.centerObject(geometry)
Map.addLayer(geometry, {'color': 'grey'}, 'Ernakulam District')


collection = s1\
    .filter(ee.Filter.date(after_start, after_end))\
    .filter(ee.Filter.bounds(geometry))

print(collection.size().getInfo())

image = ee.Image(collection.first())

# Print using pprint
pprint(image.getInfo())

image_vh = image.select('VH')

# Add the image to the map
vis_params = {
    'min': -30,
    'max': 0,
    }

Map.addLayer(image_vh, vis_params, 'VH')

2
{'bands': [{'crs': 'EPSG:32643',
            'crs_transform': [10,
                              0,
                              566060.3585340944,
                              0,
                              -10,
                              1238921.171760473],
            'data_type': {'precision': 'double', 'type': 'PixelType'},
            'dimensions': [28600, 21904],
            'id': 'VV'},
           {'crs': 'EPSG:32643',
            'crs_transform': [10,
                              0,
                              566060.3585340944,
                              0,
                              -10,
                              1238921.171760473],
            'data_type': {'precision': 'double', 'type': 'PixelType'},
            'dimensions': [28600, 21904],
            'id': 'VH'},
           {'crs': 'EPSG:32643',
            'crs_transform': [-12591.783174147247,
                              -3930.6244155707536,
                              850449.1816861514,
    

In [19]:
# Select images by predefined dates
before_start = '2018-07-15'
before_end = '2018-08-10'
after_start = '2018-08-10'
after_end = '2018-08-23'

admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
ernakulam = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Ernakulam'))
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
geometry = ernakulam.geometry()

Map = geemap.Map()
Map.centerObject(geometry)

# Define the image collection
collection = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.eq('resolution_meters', 10))\
    .filter(ee.Filter.bounds(geometry))\
    .select(['VV', 'VH'])

# Filter the collection for the defined date ranges
before_collection = collection.filter(ee.Filter.date(before_start, before_end))
after_collection = collection.filter(ee.Filter.date(after_start, after_end))

# Mosaic and clip images
before = before_collection.mosaic().clip(geometry)
after = after_collection.mosaic().clip(geometry)

before_bands = before.bandNames()
print("Before Bands:")
pprint(before_bands.getInfo())

def add_ratio_band(image):
    ratio = image.select('VH').divide(image.select('VV')).rename('VH/VV')
    return image.addBands(ratio)

before_rgb = add_ratio_band(before)
before_bands_rgb = before_rgb.bandNames()
print("Before RGB Bands:")
pprint(before_rgb.getInfo())


after_rgb = add_ratio_band(after)



before_vh = before.select('VH')
after_vh = after.select('VH')

vis_params_rgb = {'min': [-25, -25, 0], 'max': [0, 0, 2]}

Map.addLayer(before_vh, vis_params, 'Before VH')
Map.addLayer(after_vh, vis_params, 'After VH')
Map.addLayer(before_rgb, vis_params_rgb, 'Before VH/VV')
Map.addLayer(after_rgb, vis_params_rgb, 'After VH/VV')

# Change Detection RGB Visualization
# Create a RGB Composite with following Band Combination
# Before VH, After VH, Before, VH
# Hint: use ee.Image.cat() to create a 3-band image
# Display the image with a min/max range of -25 to -8
change_detection_rgb = before_vh.addBands(after_vh).addBands(before_vh)
Map.addLayer(change_detection_rgb, {'min': -25, 'max': -8}, 'Change Detection RGB 2')

# Assuming before_vh and after_vh are ee.Image objects
change_detection_rgb = ee.Image.cat([before_vh, after_vh, before_vh])
Map.addLayer(change_detection_rgb, {'min': -25, 'max': -8}, 'Change Detection RGB')


Before Bands:
['VV', 'VH']
Before RGB Bands:
{'bands': [{'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'double', 'type': 'PixelType'},
            'dimensions': [3, 2],
            'id': 'VV',
            'origin': [75, 9]},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'double', 'type': 'PixelType'},
            'dimensions': [3, 2],
            'id': 'VH',
            'origin': [75, 9]},
           {'crs': 'EPSG:4326',
            'crs_transform': [1, 0, 0, 0, 1, 0],
            'data_type': {'precision': 'double', 'type': 'PixelType'},
            'dimensions': [3, 2],
            'id': 'VH/VV',
            'origin': [75, 9]}],
 'properties': {'system:footprint': {'geometries': [{'coordinates': [76.28383080415836,
                                                                     9.83865587473557],
                                                 

In [20]:
Map

Map(center=[10.043019638012588, 76.49493184941537], controls=(WidgetControl(options=['position', 'transparent_…